# PART 03. 통계분석

## 4장. 분산분석 (ANOVA)

In [2]:
# 데이터 핸들링을 위한 패키지
import numpy as np
import pandas as pd

# 통계를 위해 사용하는 패키지
import scipy as sp
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 시각화를 위한 패키지
from matplotlib import pyplot as plt
import seaborn as sns

# %matplotlib inline 의 목적은 plt.show()함수가 호출되지 않은 경우에도 matplotlib 다이어그램을 렌더링하는 것이다.
# 그러나 현재 업데이트된 버전의 주피터 노트북 버전에서는 %matplotlib inline 를 사용하지 않더라도 Matplotlib 다이어그램을 개체로 표현한다.
# 따라서 굳이 필요하지는 않다. 그러나 코드를 깨끗하게 유지하고 자신이 만든 플롯을 호출하기 위해 여전히 관례적으로 권장된다.
%matplotlib inline

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

### 1) 일원배치 분산분석

In [24]:
df_iris = pd.read_csv( 'C:/Users/Administrator/GitHub/TIL/ADP_study/rawdata/iris.csv',
                         index_col = 'Unnamed: 0' )

df_iris.rename( columns = {'Sepal.Length':'Sepal_Length',
                           'Sepal.Width':'Sepal_Width',
                           'Petal.Length':'Petal_Length',
                           'Petal.Width':'Petal_Width'},
                inplace = True )

df_iris

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,virginica
147,6.3,2.5,5.0,1.9,virginica
148,6.5,3.0,5.2,2.0,virginica
149,6.2,3.4,5.4,2.3,virginica


In [25]:
type1 = df_one_anova[ df_iris['Species'] == 'setosa' ]['Sepal_Width']
type2 = df_one_anova[ df_iris['Species'] == 'versicolor' ]['Sepal_Width']
type3 = df_one_anova[ df_iris['Species'] == 'virginica' ]['Sepal_Width']

In [36]:
# 기본 가정 중 등분산성을 검정

## 귀무가설(H0) : 세 집단의 분산이 동일하다.
## 대립가설(H1) : 세 집단의 분산이 동일하지 않다.

# 등분산 검정 레빈(levene)
levene = stats.levene(type1, type2, type3)
print('levene result(F) : %.3f \np-value : %.3f' % (levene))

if levene[1] < 0.05:
    print("p-value < 0.05 이므로 귀무가설 기각. 등분산성을 만족하지 않는다.")
if levene[1] > 0.05:
    print("p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.")
print()

# 분산의 동질성 검정 기준 - default : 'median'. 'mean' 으로 변경 할 경우 다음과 같이 mean 으로 설정
levene = stats.levene(type1, type2, type3, center = 'mean')
print('levene result(F) : %.3f \np-value : %.3f' % (levene))
if levene[1] < 0.05:
    print("p-value < 0.05 이므로 귀무가설 기각. 등분산성을 만족하지 않는다.")
if levene[1] > 0.05:
    print("p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.")
print()

# 등분산 검정 플리그너(fligner) 
fligner = stats.fligner(type1, type2, type3)
print('fligner Result(F) : %.3f \np-value : %.3f' % (fligner))
if fligner[1] < 0.05:
    print("p-value < 0.05 이므로 귀무가설 기각. 등분산성을 만족하지 않는다.")
if fligner[1] > 0.05:
    print("p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.")
print()

# 등분산 검정 바틀렛(bartlett)
bartlett = stats.bartlett(type1, type2, type3)
print('bartlett Result(F) : %.3f \np-value : %.3f' % (bartlett))
if bartlett[1] < 0.05:
    print("p-value < 0.05 이므로 귀무가설 기각. 등분산성을 만족하지 않는다.")
if bartlett[1] > 0.05:
    print("p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.")
print()

levene result(F) : 0.590 
p-value : 0.556
p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.

levene result(F) : 0.601 
p-value : 0.550
p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.

fligner Result(F) : 0.912 
p-value : 0.634
p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.

bartlett Result(F) : 2.091 
p-value : 0.352
p-value > 0.05 이므로 귀무가설 채택. 등분산성을 만족한다.



In [13]:
## 귀무가설(H0) : 세 가지 종의 Sepal.Width의 평균은 같다.
## 대립가설(H1) : 세 가지 종의 Sepal.Width의 평균은 같지 않다.


df_one_anova = pd.DataFrame( df_iris,
                             columns = ['Sepal_Width', 'Species'] )
df_one_anova

,Sepal_Width,Species
1,3.5,setosa
2,3.0,setosa
3,3.2,setosa
4,3.1,setosa
5,3.6,setosa
...,...,...
146,3.0,virginica
147,2.5,virginica
148,3.0,virginica
149,3.4,virginica


In [10]:
# scipy.stats 패키지를 사용

type1 = df_one_anova[df_iris['Species'] == 'setosa' ]['Sepal_Width']
type2 = df_one_anova[df_iris['Species'] == 'versicolor' ]['Sepal_Width']
type3 = df_one_anova[df_iris['Species'] == 'virginica' ]['Sepal_Width']

one_anova_result = stats.f_oneway(type1, type2, type3)


print( f'F-Value: {one_anova_result[0]:.4f}    P-value: {one_anova_result[1]:.28f}' )

if one_anova_result[1] < 0.05:
    print("p-value < 0.05 이므로 귀무가설이 기각.")

print(one_anova_result)
print()

F-Value: 49.1600    P-value: 0.0000000000000000449201713331
p-value < 0.05 이므로 귀무가설이 기각.
F_onewayResult(statistic=49.160040089612075, pvalue=4.492017133309115e-17)



In [19]:
# statsmodels 패키지를 사용하여 일원분산 분석 실시

df_one_anova = pd.DataFrame( df_iris,
                         columns = ['Sepal_Width', 'Species'] )

results = ols( 'Sepal_Width ~ C(Species)', data = df_one_anova ).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Sepal_Width   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     49.16
Date:                Thu, 15 Jul 2021   Prob (F-statistic):           4.49e-17
Time:                        00:09:47   Log-Likelihood:                -49.366
No. Observations:                 150   AIC:                             104.7
Df Residuals:                     147   BIC:                             113.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    3.4280      0.048     71.359      0.000       3.333       3.523
C(Species)[T.versicolor]    -0.6580      0.068     -9.685      0.000      -0.792      -0.524
C(Species)[T.virginica]     -0.4540      0.068     -6.683      0.000      -0.588      -0.320
==============================================================================
Omnibus:                        1.920   Durbin-Watson:                   1.879
Prob(Omnibus):                  0.383   Jarque-Bera (JB):                1.632
Skew:                           0.027   Prob(JB):                        0.442
Kurtosis:                       3.508   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
# sum_sq: 편차 제곱합
# df: 자유도
# F: F통계량
# PR(>F): P-value

one_anova_table = sm.stats.anova_lm(results, typ=2)
one_anova_table

,sum_sq,df,F,PR(>F)
C(Species),11.344933,2.0,49.16004,4.492017e-17
Residual,16.962000,147.0,NaN,NaN


In [ ]:
# 데이터 셋에 대한 정보 확인
df_f_oneway.info()

In [ ]:
# 결측치가 있는지 확인
df_f_oneway.isnull().sum()

In [ ]:
# 업체별 데이터 갯수 확인
df_f_oneway['type'].value_counts()

In [ ]:
# 기술통계량 확인
df_f_oneway.describe()